# ***<span style="color:green">Create an Agent</span>***

In [ ]:
import ollama
import json


def routing(state, query, intermediate_results):
    messages = [
        {
            "role": "user",
            "content": (
                "You are a reasoning and acting agent. Based on the current state and user input, decide the next action.\n"
                f"State: {state}\n"
                f"User Input: {query}\n"
                f"Intermediate Results: {intermediate_results}\n\n"
                "Respond with one of these actions:\n"
                "- Search internal knowledges from vector database(query)\n"
                "- Do nothing\n"
                "- Search from external website(query)\n"
            )
        }
    ]
    response = ollama.chat(
        model='llama3.2:latest',
        messages=messages,

        # provide a weather checking tool to the model
        tools=[
          {
            'type': 'function',
            'function': {
              'name': 'get_greeting_conversation',
              'description': 'Get the casual greeting with user',
              'parameters': {
                'type': 'object',
                'properties': {
                  'query': {
                    "type": "string",
                    "description": "The search query string to be submitted to the search engine"
                  }
                },
                'required': ['query'],
              },
            },
          },
          {
            'type': 'function',
            'function': {
              'name': 'get_internal_knowledge',
              'description': 'Get the internal informations in Nexcel Solutions Company',
              'parameters': {
                'type': 'object',
                'properties': {
                  'query': {
                    "type": "string",
                    "description": "The search query string to be submitted to the search engine"
                  }
                },
                'required': ['query'],
              },
            },
          },
          {
            'type': 'function',
            'function': {
              'name': 'get_external_knowledge',
              'description': 'Get the external informations from websites in the internet if the internal informations is not reasonble',
              'parameters': {
                'type': 'object',
                'properties': {
                  'query': {
                    "type": "string",
                    "description": "The search query string to be submitted to the search engine"
                  }
                },
                'required': ['query'],
              },
            },
          },
          {
            'type': 'function',
            'function': {
              'name': 'do_nothing',
              'description': 'A function that performs no action',
              "parameters": {
                  "type": "object",
                  "required": [],
                  "properties": {}
              },
            },
          }
      ]
    )
    
    return response
  
  
assistant_message = routing(
    "start",
    "Hello",
    []
)
  

def process_tool_calls(response):
    # Extract tool call information from the response
    tool_calls = response.tool_calls

    if tool_calls:
        # Extract tool function details
        tool_function_name = tool_calls[0].function.name
        tool_query_string = eval(json.dumps(tool_calls[0].function.arguments))

        # Print extracted details
        print('Tool Function Name:', tool_function_name)
        print('Tool Query String:', tool_query_string)

        return tool_function_name, tool_query_string

    else:
        print("No tool calls identified.")
        return None, None
      

     
print(assistant_message.message)
process_tool_calls(assistant_message.message)

# ***<span style="color:green">Prepare Tools</span>***